In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate 

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain.agents import AgentExecutor, create_tool_calling_agent


from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import tool


In [3]:
l_llm = {v: ChatOpenAI(model=v) for v in ["gpt-4o","gpt-4o-mini"]} 

In [4]:
llm = l_llm['gpt-4o']
create_prompt = lambda system_prompt : ChatPromptTemplate.from_messages([('system',system_prompt),('human','{input}')])


In [11]:
# t = llm.invoke("What is the square root of 4?")
# tem_chain = RunnablePassthrough() | StrOutputParser()
# tem_chain.invoke(t)

In [12]:
class primaryClf(BaseModel):
    """ 
    Classify the input into one or more of the categories
    """ 
    company          : str = Field(description = "Company name in the query")
    ticker           : str = Field(description = "Ticker name of the Query")
    information_type : str = Field(description = 'Type of Information Requested. One of "company", "information_type", and "time_duration"')
    time_duration    : str = Field(description = 'Time requested in user query. Start and end date')


def get_primary_chain():

    primary_prompt = """
You are a equity research analyst. The user query will be about some information regarding a company, and you have to help identify keywords from the user query. 
You have to identify the following items
- 1) "company" : Identify the company the user is talking about. Eg "Tata Consultancy Services", "HDFC Bank", "Aditya Birla Fashion" etc
- 2) "ticker" : The NSE ticker of the identified company. Use Tavily Search to confirm. eg. "TCS.NS", "HDFCBANK.NS", "ABFRL.NS"   
- 3) "information_type" : What type of information the user has requested. The value of this key can only be one of the following "General", "Financial", or "Market". The meaning of these terms is as follows 
        "General" information refers to non-numeric information like its adress, contact information, website, industry, or company description
        "Financial" implies information regarding its balance sheet, cashflows, or PnL statement, either annual or quarterly
        "Market" refers to information regarding its share price

- 4) "time_duration" : Is the user talking about particular time-duration. Respond keeping in mind that the date today is 16-Feb-2025 .
     This should expressed be in the form <start date> - <end date>, eg a query talking about this year should be "1-Jan-2025 to 31-12-2025"
     In case of no time mentioned it should be "NA"

Your response has to be a json with these 3 keys - "company", "information_type", and "time_duration"
Do not have any other information in the response
""".strip()

    tool_search = TavilySearchResults(max_results=2)
    l_tools = [tool_search]
    llm = l_llm['gpt-4o-mini']

    llm_tools = llm.bind_tools(l_tools)
    llm_st = llm_tools.with_structured_output(primaryClf)
    chain  = create_prompt(primary_prompt) | llm_st 
    return chain



In [13]:

p_chain = get_primary_chain()

query = "Give me a description of Varun Beverages"
res = p_chain.invoke(query)
print(res)

company='Varun Beverages' ticker='VBL.NS' information_type='General' time_duration='NA'


In [ ]:
# tem_chain = RunnablePassthrough() | dict
# tem_chain.invoke(res)

{'company': 'Varun Beverages',
 'ticker': 'VBL.NS',
 'information_type': 'General',
 'time_duration': 'NA'}

In [20]:
res.json()

'{"company": "Varun Beverages", "ticker": "VBL.NS", "information_type": "General", "time_duration": "NA"}'

###  Testing YF Tool Agent

In [26]:
import yfinance as yf

In [22]:
str_keys = ['address1', 'address2', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'currency', 'lastSplitFactor', 'exchange', 'quoteType', 'symbol', 'underlyingSymbol', 'shortName', 'longName', 'timeZoneFullName', 'timeZoneShortName', 'uuid', 'messageBoardId', 'recommendationKey', 'financialCurrency']

"""
    Fetches information about the ticker mentioned. 
    argument l_fields is the set of relevant fields to chosen from the following list - {%s}
    """%(str(str_keys))


"\n    Fetches information about the ticker mentioned. \n    argument l_fields is the set of relevant fields to chosen from the following list - {['address1', 'address2', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'currency', 'lastSplitFactor', 'exchange', 'quoteType', 'symbol', 'underlyingSymbol', 'shortName', 'longName', 'timeZoneFullName', 'timeZoneShortName', 'uuid', 'messageBoardId', 'recommendationKey', 'financialCurrency']}\n    "

In [28]:
str_keys = ['address1', 'address2', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'currency', 'lastSplitFactor', 'exchange', 'quoteType', 'symbol', 'underlyingSymbol', 'shortName', 'longName', 'timeZoneFullName', 'timeZoneShortName', 'uuid', 'messageBoardId', 'recommendationKey', 'financialCurrency']


@tool
def gen_info_tool(ticker, l_fields) -> str:
    """
    Fetches information about the ticker mentioned. 
    argument l_fields is the set of relevant fields to chosen from the following list - 
    ['address1', 'address2', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'currency', 'lastSplitFactor', 'exchange', 'quoteType', 'symbol', 'underlyingSymbol', 'shortName', 'longName', 'timeZoneFullName', 'timeZoneShortName', 'uuid', 'messageBoardId', 'recommendationKey', 'financialCurrency']    "
    """
    stock = yf.Ticker(ticker)
    d_ret = {}
    for field in l_fields:
        if field in stock.info.keys():
            d_ret[field] = stock.info[field]
    return str(d_ret)

    

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You have to give response to the users query, given the following information"),
        ("ai", "{context}"),
        ("human", "Query : {input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [gen_info_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

query = "What is the full address of Varun Beverages?"

agent_executor.invoke({"input": query, "context": res.json()})

{'input': 'What is the full address of Varun Beverages?',
 'context': '{"company": "Varun Beverages", "ticker": "VBL.NS", "information_type": "General", "time_duration": "NA"}',
 'output': 'The full address of Varun Beverages is:\n\nRJ Corp House,  \nPlot No. 31 Institutional Area Sector-44,  \nGurugram,  \n122002,  \nIndia.'}